In [1]:
import pandas as pd

Crearemos un dataset personalizado para nuestra funcion PlayTimeGenre. De esta manera se mejora el rendimiento de la API

    def PlayTimeGenre( genero : str ): Debe devolver año con mas horas jugadas para dicho género.

In [2]:
#Importamos los datos limpios
df_output_steam_games=pd.read_parquet('Dataset_Limpio/output_steam_games.parquet')
df_user_items=pd.read_parquet('Dataset_Limpio/user_items.parquet')
df_user_reviews=pd.read_parquet('Dataset_Limpio/user_reviews.parquet')

In [3]:
#Para la tabla output
df_Primer=df_output_steam_games.drop(['app_name', 'developer'],axis=1)
#Para la tabla user_items
df_Segundo=df_user_items.drop(['item_name','items_count'],axis=1)
df_Primer['id']=df_Primer['id'].astype('Int64')
#Las unimos
merged_data=pd.merge(df_Segundo,df_Primer,left_on='item_id', right_on='id', how='inner')
merged_data.drop(['item_id'], axis=1, inplace=True)

In [4]:
#Transformo mi columna de array a str para poder agrupar
merged_data['genres'] = merged_data['genres'].apply(lambda x: ', '.join(x))
#Agrupamos por Id, y Genres
merged_data=merged_data.groupby(['id', 'genres']).agg({
    'playtime_forever': 'sum',
    'año_lanzamiento': 'first',
    'user_id': 'first'
}).reset_index()

In [5]:
#Dividimos la columna de generos en multiples dummies
df_dummies=merged_data['genres'].str.replace("amp;", "").str.get_dummies(', ')
df_dummies = df_dummies.rename(columns=lambda x: x.strip("'"))
#Unimos los dummies con los demas datos
df_dummies = pd.concat([df_dummies, merged_data[['playtime_forever', 'año_lanzamiento']]], axis=1)

In [6]:
df_dummies

,Action,Adventure,Animation & Modeling,Audio Production,Casual,Design & Illustration,Early Access,Education,Free to Play,Indie,...,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing,playtime_forever,año_lanzamiento
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,17107858,2000
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,960524,1999
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,756375,2003
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,154424,2001
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,726545,1999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8641,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,2,2016
8642,0,0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,2016
8643,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,44,2016
8644,1,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,182,2016


In [7]:
#Definimos las columnas que no derretiremos
columnas_no_dummies = ['playtime_forever','año_lanzamiento']

# Aplicar melt al DataFrame para quedarnos con una sola fila
df_melted = pd.melt(df_dummies, id_vars=columnas_no_dummies, var_name='dummy', value_name='valor_dummy')

In [8]:
df_melted

,playtime_forever,año_lanzamiento,dummy,valor_dummy
0,17107858,2000,Action,1
1,960524,1999,Action,1
2,756375,2003,Action,1
3,154424,2001,Action,1
4,726545,1999,Action,1
...,...,...,...,...
181561,2,2016,Web Publishing,0
181562,1,2016,Web Publishing,0
181563,44,2016,Web Publishing,0
181564,182,2016,Web Publishing,0


In [9]:
#Nos quedamos con los valores dummies positivos
df_melted=df_melted[df_melted['valor_dummy']==1]
df_melted.drop('valor_dummy', inplace=True, axis=1)
#Renombramos el encabezado
df_melted=df_melted.rename(columns={'dummy':'genre'})
df_melted

C:\Users\PC GABRIEL VERON\AppData\Local\Temp\ipykernel_5196\3035692096.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_melted.drop('valor_dummy', inplace=True, axis=1)


,playtime_forever,año_lanzamiento,genre
0,17107858,2000,Action
1,960524,1999,Action
2,756375,2003,Action
3,154424,2001,Action
4,726545,1999,Action
...,...,...,...
178924,186,2015,Web Publishing
179434,11058,2007,Web Publishing
181087,135,2016,Web Publishing
181111,1,2016,Web Publishing


In [10]:
#Hacemos una lista con los generos
generos=df_melted['genre'].unique().tolist()

In [11]:
#Generamos una lista que contenga los Años mas jugados por genero
lista=[]
for genero in generos:
    #Nos quedamos con las filas del genero de interes
    Tabla=df_melted[df_melted['genre']==genero].drop(columns=['genre'])
    max_playtime_row = Tabla[Tabla['playtime_forever'] == Tabla['playtime_forever'].max()]  # Obtenemos la fila con el valor máximo de 'playtime_forever'
    year = max_playtime_row['año_lanzamiento'].values[0]  # Obtenemos el valor de 'año_lanzamiento' para esa fila
    lista.append([genero,year])

In [12]:
#Cambiamos los encabezados de columnas y transformamos a dataframe para exportar
lista=pd.DataFrame(lista)
lista=lista.rename(columns={0:'genero', 1:'año_lanzamiento'})

Exportamos el Dataset que consumirá la fucion PlayTimeGenre:

In [13]:
import os
if not os.path.exists('../Consultas'):
    os.makedirs('../Consultas')

In [14]:
#Exportamos la lista final para que sea consumida en la API
lista.to_csv('../Consultas/Consulta1.csv', index=False)

# Dataset para UserForGenre:
def UserForGenre( genero : str ): Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.

In [15]:
#Para la tabla output
df_Primer=df_output_steam_games.drop(['app_name', 'developer'],axis=1)
#Para la tabla user_items
df_Segundo=df_user_items.drop(['item_name','items_count'],axis=1)
df_Primer['id']=df_Primer['id'].astype('Int64')
#Las unimos
merged_data=pd.merge(df_Segundo,df_Primer,left_on='item_id', right_on='id', how='inner')
merged_data.drop(['item_id'], axis=1, inplace=True)

In [16]:
#Transformo mi columna de array a str para poder agrupar
merged_data['genres'] = merged_data['genres'].apply(lambda x: ', '.join(x))
#Agrupamos por user_id, y Genres
merged_data=merged_data.groupby(['user_id', 'genres']).agg({
    'playtime_forever': 'sum',
    'año_lanzamiento': 'first',
    'id': 'first'
}).reset_index()

In [17]:
#Dividimos la columna de generos en multiples dummies
df_dummies=merged_data['genres'].str.replace("amp;", "").str.get_dummies(', ')
df_dummies = df_dummies.rename(columns=lambda x: x.strip("'"))
#Unimos los dummies con los demas datos
df_dummies = pd.concat([df_dummies, merged_data[['playtime_forever', 'año_lanzamiento', 'id', 'user_id']]], axis=1)

In [18]:
df_dummies.head(1)

,Action,Adventure,Animation & Modeling,Audio Production,Casual,Design & Illustration,Early Access,Education,Free to Play,Indie,...,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing,playtime_forever,año_lanzamiento,id,user_id
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,115035,2009,1250,--000--


In [19]:
#Definimos las columnas que no derretiremos
columnas_no_dummies = ['playtime_forever','año_lanzamiento','id','user_id' ]

# Aplicar melt al DataFrame para quedarnos con una sola fila
df_melted = pd.melt(df_dummies, id_vars=columnas_no_dummies, var_name='dummy', value_name='valor_dummy')

In [20]:
#Nos quedamos con los valores dummies positivos
df_melted=df_melted[df_melted['valor_dummy']==1]
df_melted.drop('valor_dummy', inplace=True, axis=1)
#Renombramos el encabezado
df_melted=df_melted.rename(columns={'dummy':'genre'})
df_melted

,playtime_forever,año_lanzamiento,id,user_id,genre
0,115035,2009,1250,--000--,Action
1,4037,2017,304930,--000--,Action
2,4260,2015,304050,--000--,Action
3,582,2014,238460,--000--,Action
4,1597,2012,204360,--000--,Action
...,...,...,...,...,...
35819269,37926,2012,220700,zepavil,Web Publishing
35819862,1,2007,400040,zeshirky,Web Publishing
35819922,4,2012,220700,zevlupine,Web Publishing
35820325,9,2013,227240,zilaman,Web Publishing


In [21]:
#Pasamos playtime_forever de minutos a horas
df_melted['playtime_forever']=df_melted['playtime_forever']/60

In [22]:
#Como ultimo paso generaremos una tabla con los resultados para cada genero
#Para esto iteramos por genero
respuestas=[]
for genero in generos:
    #Nos quedamos con las filas del genero de interes
    Tabla=df_melted[df_melted['genre']==genero].drop(columns=['genre'])
    #Hacemos una suma de horas para cada usuario
    suma_playtime_por_usuario = Tabla.groupby('user_id')['playtime_forever'].sum().reset_index()
    #Obtenemos el usuario con mas horas del genero
    max_user=suma_playtime_por_usuario.loc[suma_playtime_por_usuario['playtime_forever'].idxmax(), 'user_id']
    #Hacemos un dataframe con los datos de este usuario
    df_usuario = Tabla[Tabla['user_id'] == max_user]
    #Sumamos las horas de juego por Año
    df_usuario=df_usuario.groupby('año_lanzamiento')['playtime_forever'].sum().reset_index()
    lista=[]
    for indice, fila in df_usuario.iterrows():
        lista.append({'Año': fila['año_lanzamiento'], 'Horas': fila['playtime_forever']})
    diccio={f"Usuario con más horas jugadas para {genero}" : max_user, 'Horas jugadas': lista}
    respuestas.append({'genero':genero, 'respuesta': diccio})

In [23]:
#Transformamos nuestra lista en un dataframe
respuestas=pd.DataFrame(respuestas)


In [24]:
#Exportamos
respuestas.to_csv('../Consultas/Consulta2.csv', index=False)

# Dataset para UsersRecommend

def UsersRecommend( año : int ): Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)

In [25]:
#Extraemos los valores que cumplen con las condiciones
recomendados=df_user_reviews[(df_user_reviews['recommend']==True) & ((df_user_reviews['sentiment_analysis']==1) | (df_user_reviews['sentiment_analysis']==2))]

In [26]:
#Quitamos columnas innecesarias
recomendados.drop(['recommend', 'sentiment_analysis'],inplace=True, axis=1)

C:\Users\PC GABRIEL VERON\AppData\Local\Temp\ipykernel_5196\1326853051.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recomendados.drop(['recommend', 'sentiment_analysis'],inplace=True, axis=1)


In [27]:
#Hacemos lo mismo para el dataframe df_output_steam_games
output_limpio=df_output_steam_games.drop(['genres', 'developer', 'año_lanzamiento'],axis=1)

In [28]:
output_limpio.isnull().sum()

app_name    0
id          0
dtype: int64

In [29]:
#Los fucionamos
merge = pd.merge(recomendados, output_limpio, left_on='item_id', right_on='id', how='left')
#Quitamos las columnas que usamos para unir
merge.drop(['item_id', 'user_id'],inplace=True, axis=1)

In [30]:
#Controlamos los id que no corresponden a ningun app_name    
merge.isnull().sum()

año            0
app_name    6039
id          6039
dtype: int64

In [31]:
merge.dropna(inplace=True)

In [32]:
#Exportamos
merge.to_csv('../Consultas/Consulta3.csv', index=False)

# Dataset para UsersWorstDeveloper

def UsersWorstDeveloper( año : int ): Devuelve el top 3 de desarrolladoras con juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)

In [33]:
df_user_reviews.columns

Index(['item_id', 'recommend', 'user_id', 'año', 'sentiment_analysis'], dtype='object')

In [34]:
#Traemos las review que nos importan
recomendados=df_user_reviews[(df_user_reviews['recommend'] == False) & (df_user_reviews['sentiment_analysis'] == 0)]
# Eliminamos las columnas innecesarias
recomendados = recomendados.drop([ 'recommend', 'user_id','sentiment_analysis'], axis=1)

In [35]:
#Hacemos lo mismo para el dataframe df_output_steam_games
output_limpio=df_output_steam_games.drop(['genres', 'app_name', 'año_lanzamiento'],axis=1)

In [36]:
#Los fucionamos
merge = pd.merge(recomendados, output_limpio, left_on='item_id', right_on='id', how='left')
#Quitamos las columnas que usamos para unir
merge.drop(['item_id', 'id'],inplace=True, axis=1)

In [37]:
#Controlamos los id que no corresponden a ningun developer
merge.isnull().sum()

año            0
developer    680
dtype: int64

In [38]:
#Dropeamos los nulos
merge.dropna(inplace=True)

In [39]:
#Exportamos
merge.to_csv('../Consultas/Consulta4.csv', index=False)

# Dataset para sentiment_analysis

def sentiment_analysis( empresa desarrolladora : str ): Según la empresa desarrolladora, se devuelve un diccionario con el nombre de la desarrolladora como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor.

Ejemplo de retorno: {'Valve' : [Negative = 182, Neutral = 120, Positive = 278]}

In [40]:
#Eliminamos columnas innecesarias
sentimientos=df_user_reviews.drop(['recommend', 'user_id', 'año'], axis=1)

In [41]:
#Limpiamos df_output_steam_games
output_limpio = df_output_steam_games.loc[:, ['id', 'developer']]

In [42]:
#Unimos ambas tablas
merge=pd.merge(sentimientos, output_limpio, left_on='item_id', right_on='id', how='inner')
#Eliminamos las columnas que usamos para unir
merge.drop(['id','item_id'],inplace=True, axis=1)

In [43]:
#Obtenemos dummies para cada valor de sentimiento
df_dummies=pd.get_dummies(merge['sentiment_analysis'], dummy_na=False)
df_dummies=df_dummies.astype(int) #Los hacemos int para sumarlos
#Y unimos a la tabla original
df=pd.concat([df_dummies, merge], axis=1).drop(['sentiment_analysis'], axis=1)

In [44]:
#Renombramos las columnas
df.rename(columns={0:'Negative', 1:'Neutral', 2:'Positive'}, inplace=True)
#Agrupamos sumando
resultado=df.groupby('developer').agg({
    'Negative':'sum',
    'Neutral':'sum',
    'Positive':'sum'
}).reset_index()

In [45]:
#Teniendo la tabla de resultados
#La exportamos para la API
resultado.to_csv('../Consultas/Consulta5.csv', index=False)

# Dataset para recomendacion_juego

def recomendacion_juego( id de producto ): Ingresando el id de producto, deberíamos recibir una lista con 5 juegos recomendados similares al ingresado.

In [46]:
#Creamos un dataframe de todos los juegos para alimentar nuestro ML
#Nos quedamos con las columnas de interes
df_Juegos=df_output_steam_games[['genres', 'app_name', 'id']]

In [51]:
from sklearn.neighbors import NearestNeighbors

#Pasamos a str la columna genres
df_Juegos['genres']=df_Juegos['genres'].apply(lambda x: ', '.join(x))
df_Juegos.drop_duplicates(inplace=True)
#Reseteamos indices
df_Juegos.reset_index(inplace=True)

#Para poder generar los dummies
df_dummies=df_Juegos['genres'].str.replace("amp;", "").str.get_dummies(', ')

# Inicializar el modelo de Nearest Neighbors
nn_model = NearestNeighbors(n_neighbors=5, metric='cosine')  #Ajustamos a 5 vecinos y la métrica a similitud de coseno

# Transformamos a matriz
data_matrix = df_dummies.values
# Entrenamos el modelo
nn_model.fit(data_matrix)

C:\Users\PC GABRIEL VERON\AppData\Local\Temp\ipykernel_5196\1400495791.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Juegos['genres']=df_Juegos['genres'].apply(lambda x: ', '.join(x))
C:\Users\PC GABRIEL VERON\AppData\Local\Temp\ipykernel_5196\1400495791.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Juegos.drop_duplicates(inplace=True)


NearestNeighbors(metric='cosine')

In [52]:
#Ahora iteramos para obtener el dataset de salida
Salida={}
for id in df_Juegos['id']:
    app_index = df_Juegos[df_Juegos['id'] == id].index[0]
    app_vector = data_matrix[app_index].reshape(1, -1)

    # Encontrar los vecinos más cercanos
    distances, indices = nn_model.kneighbors(app_vector)
    #Los agregamos a la Salida
    for i, index in enumerate(indices.flatten()):
        if index != app_index:
            lista = df_Juegos.iloc[indices.flatten()]['app_name'].tolist()
            Salida.update({id:lista})

In [55]:
#Transformamos a DataFrame
df_Salida = pd.DataFrame(list(Salida.items()), columns=['id', 'recomendaciones'])

#Exportamos para su consumo
df_Salida.to_csv('../Consultas/Consulta6.csv', index=False)